In [1]:
%load_ext autoreload

In [3]:
%aimport data

Files already downloaded and verified
Files already downloaded and verified


In [4]:
import skorch

In [5]:
import numpy as np

In [6]:
import keras

Using TensorFlow backend.


In [7]:
import torch

In [8]:
dataset = "fashion-mnist"

In [9]:
dataset_config = {
    'size': 32,
    'channels': 1,
    'classes': 10
}

In [10]:
net = skorch.net.NeuralNetClassifier(
    module=model.Offnet(
        dataset,
        dataset_config['size'],
        dataset_config['channels'],
        dataset_config['classes'],
        total_block_number=6,
        widen_factor=5,
    ),
    batch_size=32,
    max_epochs=10,
    optimizer=torch.optim.Adam,
    callbacks=[
        ('checkpoint', skorch.callbacks.Checkpoint(target="models/fakewrn{last_epoch[epoch]}.pt", monitor=None))
    ],
    device='cuda',
    train_split=None, # Workaround for https://github.com/dnouri/skorch/issues/218
    lr=1e-04,
)
net.initialize()

<class 'skorch.net.NeuralNetClassifier'>[initialized](
  module_=WideResNet(
    (sequential): Sequential(
      (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): ResidualBlockGroup(
        (sequential): Sequential(
          (0): ResidualBlock(
            (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv1): FakeConv(
              (expand): OffConv2d(
                (score): Conv2d(16, 720, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (offset_x): Conv2d(16, 720, kernel_size=(1, 1), stride=(1, 1))
                (offset_y): Conv2d(16, 720, kernel_size=(1, 1), stride=(1, 1))
              )
              (bottleneck): OffConv2d(
                (score): Conv2d(720, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (offset_x): Conv2d(720, 80, kernel_size=(1, 1), stride=(1, 1))
                (offset_y): Conv2d(720, 80

In [11]:
def process(images):
    pad_images = np.pad(images, ((0, 0), (2, 2), (2, 2)), 'constant')
    number_of_images, height, width = pad_images.shape
    return pad_images.reshape(number_of_images, 1, height, width).astype(np.float32)  / 255.0

In [12]:
(x_train_bytes, y_train_bytes), (x_test_bytes, y_test_bytes) = keras.datasets.fashion_mnist.load_data()
x_train = process(x_train_bytes)
x_test = process(x_test_bytes)
y_train = y_train_bytes.astype(np.int64)
y_test = y_test_bytes.astype(np.int64)

In [ ]:
net.fit_loop(x_train, y_train)

Checkpoint! Saving model to models/fakewrn1.pt.
  epoch    train_loss       dur
-------  ------------  --------
      1        0.6599  487.1975
Checkpoint! Saving model to models/fakewrn3.pt.
      3        0.3615  484.9360
Checkpoint! Saving model to models/fakewrn4.pt.
      4        0.3176  485.7789
Checkpoint! Saving model to models/fakewrn5.pt.
      5        0.2841  484.7368
Checkpoint! Saving model to models/fakewrn6.pt.
      6        0.2480  483.9877
Checkpoint! Saving model to models/fakewrn7.pt.
      7        0.2171  482.9607


In [ ]:
prediction_test = net.predict(x_test)

In [ ]:
(prediction_test == y_test).mean()